In [1]:
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold

import regex as re
import time
from pathlib import Path

In [42]:
# Get API key from aistudio.google.com, provided you have a Google account
GOOGLE_AI_STUDIO = ''  # <-- input your API key
genai.configure(api_key=GOOGLE_AI_STUDIO)

In [43]:
# List all available Google AI models accessible with your API key
for m in genai.list_models():
    print(m.name, '\t\t', m.supported_generation_methods)


models/chat-bison-001 		 ['generateMessage', 'countMessageTokens']
models/text-bison-001 		 ['generateText', 'countTextTokens', 'createTunedTextModel']
models/embedding-gecko-001 		 ['embedText', 'countTextTokens']
models/gemini-1.0-pro-latest 		 ['generateContent', 'countTokens']
models/gemini-1.0-pro 		 ['generateContent', 'countTokens']
models/gemini-pro 		 ['generateContent', 'countTokens']
models/gemini-1.0-pro-001 		 ['generateContent', 'countTokens', 'createTunedModel']
models/gemini-1.0-pro-vision-latest 		 ['generateContent', 'countTokens']
models/gemini-pro-vision 		 ['generateContent', 'countTokens']
models/gemini-1.5-pro-latest 		 ['generateContent', 'countTokens']
models/gemini-1.5-pro-001 		 ['generateContent', 'countTokens', 'createCachedContent']
models/gemini-1.5-pro 		 ['generateContent', 'countTokens']
models/gemini-1.5-pro-exp-0801 		 ['generateContent', 'countTokens']
models/gemini-1.5-flash-latest 		 ['generateContent', 'countTokens']
models/gemini-1.5-flash-001 	

In [44]:
# Set up model parameters
generation_config = {
  "temperature": 0.0,
  "top_p": 1,
  "top_k": 1,
  "max_output_tokens": 128 * 1024,   # 128k; different LLMs have different values; check model documentation
}

# All safety settings are set to None, because I don't want any censorship
safety_settings={
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
}

model_name = "gemini-1.5-pro-latest"    # <-- this is supposedly the best Google model (but more expensive)
model_name = "gemini-1.5-flash-latest"  # <-- this is a fast (but less accurate) model
model_name = "gemini-1.5-pro-exp-0801"  # <-- this is currently (Aug. 2024-08) in preview mode

# Enter high-level instructions (system prompt/instruction) here (and uncomment the 'system_instruction' below)
SYSTEM_PROMPT = '''
Recognize the following kinship terms:
子	son
從兄	cousin (father's side, same surname)
'''

model = genai.GenerativeModel(model_name=model_name,
                              #system_instruction=SYSTEM_PROMPT,
                              generation_config=generation_config,
                              safety_settings=safety_settings
                             )

model

genai.GenerativeModel(
    model_name='models/gemini-1.5-pro-exp-0801',
    generation_config={'temperature': 0.0, 'top_p': 1, 'top_k': 1, 'max_output_tokens': 131072},
    safety_settings={<HarmCategory.HARM_CATEGORY_HATE_SPEECH: 8>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_HARASSMENT: 7>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: 9>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: 10>: <HarmBlockThreshold.BLOCK_NONE: 4>},
    tools=None,
    system_instruction=None,
    cached_content=None
)

### Input file to be annotated

In [24]:
#
fin = '/content/wudi1of3.txt'
source_text = Path(fin).read_text(encoding='utf-8').strip().split('#####') # The delimiter ##### has been manually added

In [25]:
# Check no. of segments and the content of the 0th segment
len(source_text), source_text[0]

(8,
 '武帝上\n\n高祖武皇帝諱裕，字德輿，小名寄奴，彭城縣綏輿里人，[1]漢高帝弟楚元王交之後也。交生紅懿侯富，富生宗正辟彊，辟彊生陽城繆侯德，德生陽城節侯安民，安民生陽城釐侯慶忌，慶忌生陽城肅侯岑，岑生宗正平，平生東武城令某，某生東萊太守景，景生明經洽，洽生博士弘，弘生瑯邪都尉悝，悝生魏定襄太守某，某生邪城令亮，亮生晉北平太守膺，膺生相國掾熙，熙生開封令旭孫。旭孫生混，始過江，居晉陵郡丹徒縣之京口里，官至武原令。混生東安太守靖，靖生郡功曹翹，是為皇考。高祖以晉哀帝興寧元年歲次癸亥三月壬寅夜生。及長，身長七尺六寸，風骨奇特。家貧，有大志，不治廉隅。事繼母以孝謹稱。\n\n初為冠軍孫無終司馬。安帝隆安三年十一月，妖賊孫恩作亂於會稽，晉朝衞將軍謝琰、前將軍劉牢之東討。牢之請高祖參府軍事。十二月，牢之至吳，而賊緣道屯結，牢之命高祖與數十人覘賊遠近。會遇賊至，眾數千人，高祖便進與戰。所將人多死，而戰意方厲，手奮長刀，所殺傷甚眾。牢之子敬宣疑高祖淹久，恐為賊所困，乃輕騎尋之。既而眾騎並至，賊乃奔退，斬獲千餘人，推鋒而進，平山陰，恩遁還入海。\n\n四年五月，恩復入會稽，殺衞將軍謝琰。十一月，劉牢之復率眾東征，恩退走。牢之屯上虞，使高祖戍句章城。句章城既卑小，戰士不盈數百人，高祖常被堅執銳，為士卒先，每戰輒摧鋒陷陣，賊乃退還浹口。于時東伐諸帥，御軍無律，士卒暴掠，甚為百姓所苦。唯高祖法令明整，所至莫不親賴焉。\n\n五年春，孫恩頻攻句章，高祖屢摧破之，恩復走入海。三月，恩北出海鹽，高祖追而翼之，築城于海鹽故治。賊日來攻城，城內兵力甚弱，高祖乃選敢死之士數百人，咸脫甲冑，執短兵，並鼓噪而出，賊震懼奪氣，因其懼而奔之，並棄甲散走，斬其大帥姚盛。雖連戰剋勝，然眾寡不敵，高祖獨深慮之。一夜，偃旗匿眾，若已遁者。明晨開門，使羸疾數人登城。賊遙問劉裕所在。曰：「夜已走矣。」賊信之，乃率眾大上。高祖乘其懈怠，奮擊，大破之。恩知城不可下，乃進向滬瀆。高祖復棄城追之。海鹽令鮑陋遣子嗣之以吳兵一千，請為前驅。高祖曰：「賊兵甚精，吳人不習戰，若前驅失利，必敗我軍。可在後為聲援。」不從。是夜，高祖多設伏兵，兼置旗鼓，然一處不過數人。明日，賊率眾萬餘迎戰。前驅既交，諸伏皆出，舉旗鳴鼓。賊謂四面有軍，乃退。嗣之追奔，為賊所沒。高祖且戰且退，賊盛，所領死傷且盡。高祖慮不免，至向伏兵處，乃止，令左右脫

In [31]:
# Check the size (no. of characters) of each segment
for seg in source_text:
    print(len(seg))

1686
1561
1405
1706
1504
1481
1509
1100


In [45]:
# Create a chat session
chat = model.start_chat(history=[])

In [47]:
# Removed "For a date-like ("era name + year + month", or "era name + year", or "year + month") character sequence (e.g., 隆安三年二月, 五年七月), treat the sequence as a single token. " from Step 1 - 2024-08-21

PROMPT = """
You are an expert in the Classical Chinese language and ancient Chinese history, especially the Liu Song dynasty (5th century). You will be annotating a text enclosed in the <text> tag at the very end of this prompt. This is a multi-step prompt, consisting of 5 steps.

=== STEPS BEGIN ===
All of the following outputs from all steps will be enclosed in a single <segment number='{idx}'> tag.

Step 1:
Please tokenize the Classical Chinese passage into word tokens (a word can be a single-character or multi-character chunk). Simply separate the tokens using a single space. Retain the punctuation marks. Retain the empty line as a visual aid. For this step output the word-tokenized text within the tag <step1>.

Step 2:
Using the output from Step 1 (tokenized text), enclosed each word token with the XML-like tag <ne> (for named entity) only if the token is a named entity (NE), such as personal name, geographic location, date, job title, etc. Do not use a tag for a non-NE. For this step output the annotated text within the tag <step2>. 

Step 3:
Using the output from Step 2, for each identified named entity within the <ne> tag, replace the <ne> tag with one of the following specific tags based on the named entity's classification:
<p> for a personal name;
<g> for a geographical name;
<o> for the name of a political office or job title;
<et> for an emperor's temple name 廟號 (e.g., 高祖);
<ep> for an emperor's posthumous name 諡號 (e.g., 武帝);
<era> for an imperial era name (e.g., 永初);
<dy> for a dynastic name (e.g., 晉, 漢);
<k> for a kinship term, such as 父, 母, 子, 女, 甥, 繼母, 從弟, 從兄, 從子, 兄子, 弟子, 伯, 叔, 舅, 祖父;
<d> for a date or time.
For this step output the annotated text within the tag <step3>.

Step 4:
Use the the output from Step 3, for each named entity identified as a personal name (<p>), attempt to recover the full name associated with this named entity by adding the surname; place this full name as an attribute to the <p> tag. For example, <p fn="劉裕">裕<p/>, who is the founding emperor of the Liu Song Dynasty 劉裕 with the surnamed 劉. Output the entire annotated text. For a named entity identified as a date, if it is a year, attempt to prefix it with the proper era name, and place this full year designation (e.g., 永初二年, 隆安三年) as an attribute to the <d> tag, e.g., <d fd='永初二年'>二年</d>. Output the annotated text within the tag <step4>.

Step 5:
Provide a summary of the annotations made at the end within a <summary> tag.

=== STEPS END ===

The text to be annotated is:
<text>
{text_to_be_annotated}
</text>
"""

SLEEP_TIME = 30

fon = '/content/wudi1of3_annotated.Gemini1.5Pro.Exp0801.txt'

with open(fon, 'a', encoding='utf-8', newline='\n') as fo:

    for idx, src in enumerate(source_text[0:]):
        #if idx < 7: continue
        if src.startswith('#'): continue  # skip the #### lines
        text_to_be_annotated = src.strip()

        prompt = PROMPT.format(text_to_be_annotated=text_to_be_annotated, idx=idx)
        print(f'Segment {idx}:') 
        print(prompt)

        if idx % 2 == 0:  # clear history for every two chat.send_message() calls 
            chat = model.start_chat(history=[])
        
        start_time = time.time()
        response = chat.send_message(prompt)
        for chunk in response:
            fo.write(chunk.text + '\n')
            #fo.write("_"*80 + '\n')
        fo.flush()
        
        response = chat.send_message('Please continue.')  # if there are too many output tokens, we might need this prompt to force the model to complete the text-generation process
        for chunk in response:
            fo.write(chunk.text + '\n')
            fo.write("_"*80 + '\n')
        fo.flush()
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"Elapsed time for Segment {idx}: {elapsed_time:.2f} seconds")
        
        print('='*25)
        print(f'**** Sleeping {SLEEP_TIME} seconds....')
        time.sleep(SLEEP_TIME)
        print('**** Now continuing...')

print("\n\n***** ALL DONE!!!")

Segment 7:

You are an expert in the Classical Chinese language and ancient Chinese history, especially the Liu Song dynasty (5th century). You will be annotating a text enclosed in the <text> tag at the very end of this prompt. This is a multi-step prompt, consisting of 5 steps.

=== STEPS BEGIN ===
All of the following outputs from all steps will be enclosed in a single <segment number='7'> tag.

Step 1:
Please tokenize the Classical Chinese passage into word tokens (a word can be a single-character or multi-character chunk). Simply separate the tokens using a single space. Retain the punctuation marks. Retain the empty line as a visual aid. For this step output the word-tokenized text within the tag <step1>.

Step 2:
Using the output from Step 1 (tokenized text), enclosed each word token with the XML-like tag <ne> (for named entity) only if the token is a named entity (NE), such as personal name, geographic location, date, job title, etc. Do not use a tag for a non-NE. For this step

In [18]:
chat

ChatSession(
    model=genai.GenerativeModel(
        model_name='models/gemini-1.5-pro-exp-0801',
        generation_config={'temperature': 0.0, 'top_p': 1, 'top_k': 1, 'max_output_tokens': 131072},
        safety_settings={<HarmCategory.HARM_CATEGORY_HATE_SPEECH: 8>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_HARASSMENT: 7>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: 9>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: 10>: <HarmBlockThreshold.BLOCK_NONE: 4>},
        tools=None,
        system_instruction="\nRecognize the following kinship terms:\n子\tson\n從兄\tcousin (father's side, same surname)\n",
        cached_content=None
    ),
    history=[protos.Content({'parts': [{'text': '\nYou are an...故。\n</text>\n'}], 'role': 'user'}), protos.Content({'parts': [{'text': '```xml\n<seg...東海</g> <p>何無忌'}], 'role': 'model'}), protos.Content({'parts': [{'text': '\nYou are an...故。\n</text>\n'}], 'ro